In [ ]:
import pandas as pd
from tabulate import tabulate # Tener encuenta que si no tienes la librería tabulate debes instalarla con pip install tabulate

In [ ]:
## Colocar la ruta de los archivos de la prueba
ruta_base = r"C:\Users\ANDECUMO\Documents\01 INFORMACIÓN PERSONAL\prueba\Prueba 5\Prueba\Data"

In [3]:
## Leer los archivos CSV con la codificación 'latin1'
empresa_df = pd.read_csv(ruta_base + r"\Empresa.csv", sep=';', encoding='latin1')
persona_df = pd.read_csv(ruta_base + r"\Persona.csv", sep=';', encoding='latin1')
consumo_df = pd.read_csv(ruta_base + r"\Consumo.csv", sep=';', encoding='latin1')
## Realice cambios en la  columna 'Valor' a tipo float, reemplazando comas por puntos

consumo_df['Valor'] = consumo_df['Valor'].astype(str).str.replace(',', '.').astype(float)

In [ ]:
print("¿Existen temporadas de mayor venta de productos?")

# realice la agrupación de "periodo, ues, producto" 
ventas_por_periodo = consumo_df.groupby(['Periodo', 'UES', 'Producto'])['Valor'].sum().reset_index()

# Encontrar el índice de la fila con la venta máxima por periodo y UES
indices_maximos = ventas_por_periodo.groupby(['Periodo', 'UES'])['Valor'].idxmax()

# Obtener las filas con las ventas máximas por periodo y UES
ventas_maximas_por_periodo = ventas_por_periodo.loc[indices_maximos]

# Ordenar el DataFrame por 'Periodo' y 'UES' para una mejor presentación
ventas_maximas_por_periodo = ventas_maximas_por_periodo.sort_values(by=['Periodo', 'UES'], ascending=False)

# Formatear la columna 'Valor' como moneda
ventas_maximas_por_periodo['Valor'] = ventas_maximas_por_periodo['Valor'].map(lambda x: f"${x:,.2f}")

print(ventas_maximas_por_periodo.to_string(index=False))


In [ ]:
print("cual es la participación de consumo de personas afiliadas y no afiliadas")
# Rellenar los valores nulos en la columna 'UES' con "SIN UES"
consumo_df['UES'] = consumo_df['UES'].fillna("SIN UES")

# Personas afiliadas: están en consumo_df y en persona_df
personas_afiliadas = consumo_df[consumo_df['NumIdPersona'].isin(persona_df['NumIdPersona'])]['NumIdPersona'].unique()
#  Personas no afiliadas: están en consumo_df pero no en persona_df
personas_no_afiliadas = consumo_df[~consumo_df['NumIdPersona'].isin(persona_df['NumIdPersona'])]['NumIdPersona'].unique()

# Calcular el total de personas que consumieron
total_personas_consumo = consumo_df['NumIdPersona'].nunique()

# Calcular los porcentajes
porcentaje_afiliadas = (len(personas_afiliadas) / total_personas_consumo) * 100
porcentaje_no_afiliadas = (len(personas_no_afiliadas) / total_personas_consumo) * 100

# Resultados de afiliados y no afiliados 
print(f"Porcentaje de personas afiliadas que consumieron: {porcentaje_afiliadas:.2f}%")
print(f"Porcentaje de personas no afiliadas que consumieron: {porcentaje_no_afiliadas:.2f}%")


In [ ]:
print("¿Cuál es el consumo total por unidad de negocio?")

# Calcular el consumo total por UES
consumo_total_por_ues = consumo_df.groupby('UES')['Valor'].sum().reset_index()

# Ordenar el DataFrame por la columna 'Valor' de mayor a menor
consumo_total_por_ues = consumo_total_por_ues.sort_values(by='Valor', ascending=False)

# Formatear la columna 'Valor' como moneda
consumo_total_por_ues['Valor'] = consumo_total_por_ues['Valor'].map(lambda x: f"${x:,.2f}")
# Imprimir el resultado en formato tabulado
print("Estos son los valores de las unidades de negocio (UES):")
print(consumo_total_por_ues.to_string(index=False))

In [ ]:
print("¿Cuáles son las unidades y productos de mayor uso en cada categoría??")

# agrupamos UES y Producto, y contar la frecuencia
uso_por_ues_producto = consumo_df.groupby(['UES', 'Producto']).size().reset_index(name='Frecuencia')

# total total de registros por UES
total_por_ues = consumo_df.groupby('UES').size().reset_index(name='Total_UES')

# DataFrames para calcular el porcentaje de frecuencia por UES
uso_por_ues_producto = pd.merge(uso_por_ues_producto, total_por_ues, on='UES', how='left')
uso_por_ues_producto['% Mayor-Uso'] = (uso_por_ues_producto['Frecuencia'] / uso_por_ues_producto['Total_UES']) * 100

# Redondear a 3 decimales
uso_por_ues_producto['% Mayor-Uso'] = uso_por_ues_producto['% Mayor-Uso'].round(3)

# Identificar el producto más usado por cada UES, solo dejar valores unicos.
uso_por_ues_producto_sorted = uso_por_ues_producto.sort_values(by='Frecuencia', ascending=False)
producto_mas_usado_por_ues = uso_por_ues_producto_sorted.drop_duplicates(subset=['UES'])

#salida como una tabla
tabla_resultados = producto_mas_usado_por_ues[['UES', 'Producto', '% Mayor-Uso']]
tabla_resultados.columns = ['UES', 'Producto', '% Mayor-Uso'] 

# Mostrar la tabla
print(tabulate(tabla_resultados, headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
print ("Identifique los clientes (afiliados y no afiliados) con mayor frecuencia de uso y mayor valor neto de venta.")

# Identificar si un cliente es afiliado o no afiliado
consumo_df['Tipo_Cliente'] = consumo_df['NumIdPersona'].isin(persona_df['NumIdPersona']).map({True: 'Afiliado', False: 'No Afiliado'})

# Calcular la frecuencia de uso y el valor neto de venta por cliente
clientes_analisis = consumo_df.groupby(['NumIdPersona', 'Tipo_Cliente']).agg(
    Frecuencia=('Valor', 'size'), 
    Valor_Neto=('Valor', 'sum')   
).reset_index()

# Identificar los clientes con mayor frecuencia de uso
clientes_mayor_frecuencia = clientes_analisis.sort_values(by='Frecuencia', ascending=False).head(30)

# Identificar los clientes con mayor valor neto de venta
clientes_mayor_valor = clientes_analisis.sort_values(by='Valor_Neto', ascending=False).head(30)

# convertir valor en moneda
clientes_mayor_frecuencia['Valor_Neto'] = clientes_mayor_frecuencia['Valor_Neto'].map(lambda x: f"${x:,.2f}")
clientes_mayor_valor['Valor_Neto'] = clientes_mayor_valor['Valor_Neto'].map(lambda x: f"${x:,.2f}")

# Mistrar los reultados
print("Clientes con mayor frecuencia de uso:")
print(tabulate(clientes_mayor_frecuencia, headers='keys', tablefmt='pretty', showindex=False))

print("\nClientes con mayor valor neto de venta (ordenados de mayor a menor):")
print(tabulate(clientes_mayor_valor, headers='keys', tablefmt='pretty', showindex=False))